In [1]:
!pwd

/scratch/lgrinszt/lm_tab/scripts


In [2]:
%load_ext autoreload
%autoreload 2

In [15]:
from src.data_loading import load_data
from src.utils import FeaturesExtractor, FixedSizeSplit
from src.encodings import encode, encode_high_cardinality_features
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [58]:
import submitit
from functools import partial
from tqdm import tqdm
encodings = ["skrub__minhash_30"]#, "lm__all-distilroberta-v1", "lm__all-mpnet-base-v2", "openai__"]
#encodings = ["lm__all-distilroberta-v1"]

# model_names = [
#     "BAAI/bge-large-en-v1.5",
#     "BAAI/bge-base-en-v1.5",
#     "llmrails/ember-v1",
#     "thenlper/gte-large",
#     "thenlper/gte-base",
#     "intfloat/e5-large-v2",
#     "BAAI/bge-small-en-v1.5",
#     "hkunlp/instructor-xl",
#     "hkunlp/instructor-large",
#     "intfloat/e5-base-v2",
#     "intfloat/multilingual-e5-large",
#     "intfloat/e5-large",
#     "thenlper/gte-small",
#     "intfloat/e5-base",
#     "intfloat/e5-small-v2",
#     "hkunlp/instructor-base",
#     "sentence-t5-xxl",
#     "intfloat/multilingual-e5-base",
#     #"XLM-3B5-embedding",
#     "gtr-t5-xxl",
#     #"SGPT-5.8B-weightedmean-msmarco-specb-bitfit",
#     "intfloat/e5-small",
#     "TaylorAI/gte-tiny",
#     "gtr-t5-xl",
#     "gtr-t5-large",
#     #"XLM-0B6-embedding",
#     "intfloat/multilingual-e5-small",
#     "sentence-t5-xl",
#     "all-mpnet-base-v2",
#     #"sgpt-bloom-7b1-msmarco",
#     "jinaai/jina-embedding-l-en-v1",
#     #"SGPT-2.7B-weightedmean-msmarco-specb-bitfit",
#     "sentence-t5-large",
#     #"MegatronBert-1B3-embedding",
#     "TaylorAI/bge-micro-v2",
#     "all-MiniLM-L12-v2",
#     "all-MiniLM-L6-v2",
#     "jinaai/jina-embedding-b-en-v1",
#     #"SGPT-1.3B-weightedmean-msmarco-specb-bitfit",
#     "gtr-t5-base",
#     "nthakur/contriever-base-msmarco",
#     "TaylorAI/bge-micro",
#     "sentence-t5-base",
#     "paraphrase-multilingual-mpnet-base-v2",
#     "Hum-Works/lodestone-base-4096-v1",
#     #"SGPT-5.8B-weightedmean-nli-bitfit",
#     "paraphrase-multilingual-MiniLM-L12-v2",
#     "msmarco-bert-co-condensor",
#     "jinaai/jina-embedding-s-en-v1"
# ]
# for model_name in model_names:
#     encodings.append(f"lm__{model_name}")


# encodings.extend(["skrub__minhash_30_word_none", "skrub__minhash_30_tokenizer_gpt2"])

#encodings = ["lm__all-distilroberta-v1", "lm__all-mpnet-base-v2"]

#datasets = ["journal_jcr_cls", "movies", "michelin", "spotify", "employee_salary", "museums", "fifa_footballplayers_22", "jp_anime", "clear_corpus", "company_employees", "us_presidential", "us_accidents_severity", "us_accidents_counts", "wine_review"]
#datasets.extend(["building_permits", "agora", "met_objects", "public", "kickstarter", "colleges", "drug_directory", "medical_charge", "traffic_violations"])
#datasets = ["drug_directory", "met_objects"]
datasets = ["agora"]


executor = submitit.AutoExecutor(folder="logs")
executor.update_parameters(timeout_min=400, slurm_partition='parietal,normal', slurm_array_parallelism=300, cpus_per_task=2,
                            exclude="margpu009")


jobs = []

for dataset in tqdm(datasets):
    for encoding in tqdm(encodings, leave=False):
        X, y = load_data(dataset, max_rows=10000)
        #X_enc = encode(X_text, encoding, dataset_name=dataset, override_cache=False)
        X_enc, X_rest = encode_high_cardinality_features(X, encoding, dataset_name=dataset, override_cache=False, cardinality_threshold=30)

/scratch/lgrinszt/micromamba/envs/dpo/lib/python3.10/site-packages/submitit/auto/auto.py:23: UserWarning: Setting 'exclude' is deprecated. Use 'slurm_exclude' instead.
  warnings.warn(f"Setting '{arg}' is deprecated. Use '{new_arg}' instead.")
  0%|          | 0/1 [00:00<?, ?it/s]

Removed 1 columns with missing values on 8 columns
Removed 37355 rows with missing values on 85572 rows
Removed 37355 rows with missing values on 48217 rows
Removed 1 columns with missing values on 6 columns
New shape: (48217, 7)
Original task: regression for agora
Converting to binary classification
Classes (array([0, 1]), array([24113, 24104]))
X shape: (10000, 7), y shape: (10000,)


100%|██████████| 1/1 [00:01<00:00,  1.54s/it]

numeric ['Price']
high_card_cat ['Vendor', 'Category', 'Item', 'Origin', 'Destination', 'name']
High cardinality columns ['Vendor', 'Category', 'Item', 'Origin', 'Destination', 'name']
working dir /scratch/lgrinszt/lm_tab/scripts
Loaded from cache
working dir /scratch/lgrinszt/lm_tab/scripts
Loaded from cache
working dir /scratch/lgrinszt/lm_tab/scripts
Loaded from cache
working dir /scratch/lgrinszt/lm_tab/scripts
Loaded from cache
working dir /scratch/lgrinszt/lm_tab/scripts
Loaded from cache
working dir /scratch/lgrinszt/lm_tab/scripts
Loaded from cache


In [59]:
# compute number of unique elements per column
for col in X.columns:
    print(f"{col}: {len(X[col].unique())}")

Vendor: 1422
Category: 93
Item: 9988
Price: 9884
Origin: 186
Destination: 532
name: 9988


In [60]:
X_enc

,Vendor__0,Vendor__1,Vendor__2,Vendor__3,Vendor__4,Vendor__5,Vendor__6,Vendor__7,Vendor__8,Vendor__9,...,name__20,name__21,name__22,name__23,name__24,name__25,name__26,name__27,name__28,name__29
0,-2.126502e+09,-1.950527e+09,-1.928854e+09,-2.130174e+09,-1.956612e+09,-1.930518e+09,-2.084143e+09,-2.131721e+09,-2.096258e+09,-2.087500e+09,...,-2.100470e+09,-2.112149e+09,-2.105834e+09,-2.125058e+09,-2.139867e+09,-1.897039e+09,-1.982875e+09,-2.136507e+09,-2.121027e+09,-2.013919e+09
1,-1.287892e+09,-2.106274e+09,-3.778202e+08,-1.258231e+09,-1.537158e+09,-1.228189e+09,-1.777688e+09,-1.864403e+09,-1.518832e+09,-9.232291e+08,...,-2.139064e+09,-2.134909e+09,-1.969169e+09,-1.920846e+09,-1.719875e+09,-2.081191e+09,-2.106766e+09,-2.129676e+09,-1.812853e+09,-2.089781e+09
2,-1.021094e+09,-2.110522e+09,-2.097152e+09,-1.434570e+09,-1.891845e+09,-1.000239e+09,-2.000371e+09,-2.101812e+09,-1.465632e+09,-2.122120e+09,...,-2.111383e+09,-2.143950e+09,-2.139554e+09,-2.129652e+09,-2.086118e+09,-2.021854e+09,-2.117788e+09,-2.071402e+09,-1.902590e+09,-2.133570e+09
3,-2.085421e+09,-2.106274e+09,-2.042901e+09,-1.864994e+09,-1.983355e+09,-1.818518e+09,-1.808833e+09,-2.098677e+09,-2.065360e+09,-1.966892e+09,...,-2.039056e+09,-2.146975e+09,-2.056534e+09,-1.934834e+09,-2.131045e+09,-2.072786e+09,-2.136662e+09,-2.091219e+09,-2.137393e+09,-2.113143e+09
4,-2.008292e+09,-1.877980e+09,-1.989600e+09,-1.899041e+09,-2.052206e+09,-1.876049e+09,-1.985287e+09,-1.918418e+09,-2.110388e+09,-2.090792e+09,...,-2.115323e+09,-2.112149e+09,-2.095156e+09,-2.074471e+09,-2.139867e+09,-2.103609e+09,-2.107688e+09,-2.125669e+09,-2.073335e+09,-2.099509e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-1.967212e+09,-2.137638e+09,-1.786618e+09,-1.998618e+09,-1.860407e+09,-1.799890e+09,-1.461668e+09,-1.418325e+09,-2.000461e+09,-1.689187e+09,...,-2.114070e+09,-2.129906e+09,-2.134135e+09,-2.124372e+09,-2.071324e+09,-2.111948e+09,-2.063745e+09,-2.114560e+09,-2.103723e+09,-2.070772e+09
9996,-1.737019e+09,-1.933067e+09,-1.986280e+09,-2.097068e+09,-1.854814e+09,-2.027841e+09,-2.134156e+09,-2.063007e+09,-1.995449e+09,-1.745134e+09,...,-2.137358e+09,-2.143422e+09,-2.118717e+09,-2.129652e+09,-2.111954e+09,-2.080744e+09,-2.105423e+09,-2.077050e+09,-2.140568e+09,-2.082163e+09
9997,-2.029614e+09,-1.241393e+09,-1.915339e+09,-1.304476e+09,-1.953826e+09,-1.976031e+09,-1.723939e+09,-1.635422e+09,-1.833757e+09,-1.896639e+09,...,-2.080987e+09,-2.143422e+09,-2.140968e+09,-2.146928e+09,-2.131331e+09,-2.130327e+09,-2.122076e+09,-2.109461e+09,-2.065652e+09,-2.109453e+09
9998,-2.141150e+09,-1.421928e+09,-2.049156e+09,-2.075022e+09,-1.860407e+09,-1.705955e+09,-2.126412e+09,-1.796863e+09,-1.775558e+09,-1.137585e+09,...,-2.126585e+09,-2.112149e+09,-2.092722e+09,-2.129652e+09,-2.137029e+09,-2.077397e+09,-1.886511e+09,-2.057681e+09,-1.640868e+09,-1.992615e+09


In [61]:
X_rest

,Price
47119,0.524476
14481,0.354743
4413,0.166647
38609,0.590000
11032,0.082615
...,...
26561,0.292267
12375,0.300355
9288,0.648855
17884,0.176550


In [39]:
from src.utils import run_on_encoded_data
from sklearn.ensemble import GradientBoostingClassifier
cv = FixedSizeSplit(n_splits=5, n_train=1000, random_state=42)
run_on_encoded_data(X_enc, X_rest, y, "pca", PCA(n_components=30), "GradientBoostingClassifier", GradientBoostingClassifier(), encoding,
                        cv, regression=False)

Running GradientBoostingClassifier with pca and lm__all-distilroberta-v1
[[768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 829, 830, 831, 832, 833, 834, 835, 836, 837, 838, 839, 840, 841, 842, 843, 844, 845, 846, 847, 848, 849, 850, 851, 852, 853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 883, 884, 885, 886, 887, 888, 889, 890, 891, 892, 893, 894, 895, 896, 897, 898, 899, 900, 901, 902, 903, 904, 905, 906, 907, 908, 909, 910, 911, 912, 913, 914, 915, 916, 917, 918, 919, 920, 921, 922, 923, 924, 925, 926, 927, 928, 929, 930, 931, 932, 933, 934, 935, 936, 937, 938, 939, 940, 941, 942, 943, 944, 945, 946, 947, 948, 949, 950, 951, 952, 

{'encoding': 'lm__all-distilroberta-v1',
 'dim_reduction': 'pca',
 'model': 'GradientBoostingClassifier',
 'n_train': 1000,
 'n_test': None,
 'accuracies': array([0.66666667, 0.678     , 0.67355556, 0.68377778, 0.663     ]),
 'roc_auc': array([0.73490493, 0.74537754, 0.74267632, 0.74673211, 0.7293735 ])}

In [44]:
X_rest

,Price
47119,0.524476
14481,0.354743
4413,0.166647
38609,0.590000
11032,0.082615
...,...
26561,0.292267
12375,0.300355
9288,0.648855
17884,0.176550


In [22]:
5 * 30

150

In [71]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd

#previous_results = pd.read_csv("../results/results_all_01_10.csv")
#new_results = pd.read_csv("../results/results_all_openai.csv")
#skrub_results = pd.read_csv("../results/results_all_skrub.csv")
#melted_results = pd.concat([previous_results, new_results, skrub_results])
#melted_results = pd.read_csv("../results/results_all_04_10.csv")
#melted_results = pd.read_csv("../results/results_all_04_10_5000.csv")
#melted_results["accuracy"] = melted_results["roc_auc"]
melted_results = pd.read_csv("../results/results_18_10.csv")

In [62]:
melted_results[(melted_results["dataset"] == "agora") & (melted_results["encoding"] == "skrub__minhash_30") & (melted_results["model"] == "GradientBoostingClassifier") &  (melted_results["features"] == "all")]

,encoding,dim_reduction,model,accuracies,roc_auc,n_train,n_test,dataset,features
105305,skrub__minhash_30,passthrough,GradientBoostingClassifier,0.6380,0.686041,1000,2000,agora,all
105306,skrub__minhash_30,passthrough,GradientBoostingClassifier,0.6265,0.685233,1000,2000,agora,all
105307,skrub__minhash_30,passthrough,GradientBoostingClassifier,0.6460,0.701745,1000,2000,agora,all
105308,skrub__minhash_30,passthrough,GradientBoostingClassifier,0.6425,0.696278,1000,2000,agora,all
105309,skrub__minhash_30,passthrough,GradientBoostingClassifier,0.6395,0.685612,1000,2000,agora,all
105335,skrub__minhash_30,passthrough,GradientBoostingClassifier,0.6765,0.733275,2000,2000,agora,all
105336,skrub__minhash_30,passthrough,GradientBoostingClassifier,0.6795,0.747361,2000,2000,agora,all
105337,skrub__minhash_30,passthrough,GradientBoostingClassifier,0.6910,0.764034,2000,2000,agora,all
105338,skrub__minhash_30,passthrough,GradientBoostingClassifier,0.6880,0.763236,2000,2000,agora,all
105339,skrub__minhash_30,passthrough,GradientBoostingClassifier,0.6940,0.747164,2000,2000,agora,all


In [72]:
melted_results["accuracy"] = melted_results["accuracies"]

In [73]:
import plotly.express as px

# filter for models starting by "skrub" or openai or "lm__all-distilroberta-v1", "lm__all-mpnet-base-v2"
melted_results = melted_results[(melted_results["dataset"] == "agora") & (melted_results["encoding"] == "skrub__minhash_30") & (melted_results["model"] == "GradientBoostingClassifier")]
group_cols = ['dataset', 'model', 'dim_reduction', 'encoding', "features", "n_train"]
# show counts per group
print(melted_results.groupby(group_cols).count())
melted_results = melted_results.groupby(group_cols).mean().reset_index()
print(melted_results.shape)




# # Create the plot
fig = px.strip(
    data_frame=melted_results,
    x="accuracy",
    y="dataset",
    color="n_train",
    #color="dim_reduction",
    facet_row = "features",
    category_orders={"model": ['TabPFNClassifier', 'GradientBoostingClassifier', 'LogisticRegression']},
    title="Normalized accuracy across datasets",
    hover_data=melted_results.columns,
    labels={"accuracy": "Accuracy", "dataset": "Dataset", "model": "Model"},
    height=1200,
    width=900,
)

# Show the figure
fig.show()


                                                                                      accuracies  \
dataset model                      dim_reduction encoding          features  n_train               
agora   GradientBoostingClassifier passthrough   skrub__minhash_30 all       1000              5   
                                                                             2000              5   
                                                                             3000              5   
                                                                             4000              5   
                                                                             5000              5   
                                                                   rest_only 1000              5   
                                                                             2000              5   
                                                                             3000              5   
